In [2]:
import pandas as pd
import os

In [8]:
test = pd.read_csv('report_data/' + 'A.csv').drop(columns=['date'])
test['fiscalDateEnding'] = pd.to_datetime(test['fiscalDateEnding'])
test['reportedDate'] = pd.to_datetime(test['reportedDate'])
# add a column containing the stock name
test['stock'] = 'A'
# get the dtypes of the columns
test

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,stock
0,2023-01-31,2023-02-28,1.37,1.30,0.07,5.3846,A
1,2022-10-31,2022-11-21,1.53,1.39,0.14,10.0719,A
2,2022-07-31,2022-08-16,1.34,1.20,0.14,11.6667,A
3,2022-04-30,2022-05-24,1.13,1.12,0.01,0.8929,A
4,2022-01-31,2022-02-22,1.21,1.18,0.03,2.5424,A
...,...,...,...,...,...,...,...
88,2001-01-31,2001-02-20,0.51,0.45,0.06,13.3333,A
89,2000-10-31,2000-11-20,0.73,0.54,0.19,35.1852,A
90,2000-07-31,2000-08-17,0.39,0.20,0.19,95.0000,A
91,2000-04-30,2000-05-16,0.35,0.32,0.03,9.3750,A


In [15]:
dataset = pd.DataFrame()
# load a list of earnings reports in csv and create a dataframe
for earning in os.listdir('report_data'):
    earnings_df = pd.read_csv('report_data/' + earning).drop(columns=['date'], errors='ignore')
    # remove a column from the dataframe
    earnings_df['fiscalDateEnding'] = pd.to_datetime(earnings_df['fiscalDateEnding'])    
    earnings_df['reportedDate'] = pd.to_datetime(earnings_df['reportedDate'])
    earnings_df['stock'] = earning.split('.')[0]
    # only take the first 10 rows
    dataset = dataset.append(earnings_df.head(10), ignore_index=True)

C:\Users\David\AppData\Local\Temp\ipykernel_24968\1229207121.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(earnings_df.head(10), ignore_index=True)
C:\Users\David\AppData\Local\Temp\ipykernel_24968\1229207121.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(earnings_df.head(10), ignore_index=True)
C:\Users\David\AppData\Local\Temp\ipykernel_24968\1229207121.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(earnings_df.head(10), ignore_index=True)
C:\Users\David\AppData\Local\Temp\ipykernel_24968\1229207121.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

In [16]:
dataset

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,stock
0,2023-01-31,2023-02-28,1.37,1.3,0.07,5.3846,A
1,2022-10-31,2022-11-21,1.53,1.39,0.14,10.0719,A
2,2022-07-31,2022-08-16,1.34,1.2,0.14,11.6667,A
3,2022-04-30,2022-05-24,1.13,1.12,0.01,0.8929,A
4,2022-01-31,2022-02-22,1.21,1.18,0.03,2.5424,A
...,...,...,...,...,...,...,...
3913,2021-09-30,2021-11-04,1.25,1.1,0.15,13.6364,ZTS
3914,2021-06-30,2021-08-05,1.19,1.08,0.11,10.1852,ZTS
3915,2021-03-31,2021-05-06,1.26,1.03,0.23,22.3301,ZTS
3916,2020-12-31,2021-02-16,0.91,0.87,0.04,4.5977,ZTS


In [17]:
import pymysql
from sqlalchemy import create_engine
import sqlalchemy

rds_key_pass = pd.read_csv('db_key_pass.csv')
username = rds_key_pass['ID'][0]
password = rds_key_pass['PASS'][0]
url = rds_key_pass['URL'][0]

# Establish connection to the MySQL database
conn = pymysql.connect(host=url, user=username, password=password, db='stock_data')

# Create a SQLAlchemy engine object
engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)

# Convert the pandas DataFrame to a MySQL table
dataset.to_sql(name='Earnings', con=engine, if_exists='append', index=False, dtype={'fiscalDateEnding': sqlalchemy.types.DATETIME, 'reportedDate': sqlalchemy.types.DATETIME})

# Close the connection
conn.close()